In [1]:
#chack for updated files\n,
%load_ext autoreload
%autoreload 2
#import packages
from __future__ import division

import numpy as np
import glob, json
import pickle
import os as os_pack
import matplotlib.pyplot as plt
import corner
#%matplotlib inline\n",
%config InlineBackend.figure_format = 'retina'
import healpy as hp
import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl
import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise_extensions import blocks
from enterprise_extensions import models as ee_models
from enterprise_extensions import model_utils as ee_model_utils
from enterprise_extensions import model_orfs
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise_extensions import sampler as ee_sampler
from enterprise.signals.signal_base import LogLikelihood
import enterprise_wavelets as models
from enterprise.signals.deterministic_signals import Deterministic
from enterprise.signals.parameter import function
from la_forge.core import Core
from la_forge.diagnostics import plot_chains
from la_forge import rednoise
import la_forge
import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
    #style

In [20]:
#Loading in pickle and noise files
pint_pickle = '/home/reyna/OS_15yr/15yr_data/v1p1_de440_pint_bipm2019.pkl'
noise_file = '/home/reyna/OS_15yr/15yr_data/v1p1_wn_dict.json'
#psrlist = np.loadtxt('/home/reyna/15yr_v1p0/15yr_v1-20211001T235643Z-001/15yr_v1/psrlist_15yr_pint.txt', dtype = str)
with open(noise_file, 'r') as h:
    noise_params = json.load(h)
with open(pint_pickle,'rb') as f:
    allpsrs = pickle.load(f)
psrs = []
for ii,p in enumerate(allpsrs):
    psrs.append(p)
#Temporary to get code to not crash
psrs = psrs[0:4]
psrlist = [psr.name for psr in psrs]
print(psrlist)
for i in range(len(psrs)):
    print((max(psrs[i].toas) - min(psrs[i].toas))/(3.17*10**(7)))

['B1855+09', 'B1937+21', 'B1953+29', 'J0023+0923']
15.515060297068947
15.802872967471835
11.08458774068679
8.97542529762611


In [21]:
glitches = []
log10_f0 = parameter.Uniform(np.log10(3.5e-9), np.log10(1e-7))("Glitch_"+str(1)+'_'+'log10_f0')
phase0 = parameter.Uniform(0, 2*np.pi)("Glitch_"+str(1)+'_'+'phase0')
tau = parameter.Uniform(0.2, 5)("Glitch_"+str(1)+'_'+'tau')
t0 = parameter.Uniform(0.0, 10.0)("Glitch_"+str(1)+'_'+'t0')
psr_idx = parameter.Uniform(-0.5, len(psrs)-0.5)("Glitch_"+str(1)+'_'+'psr_idx')
log10_h = parameter.LinearExp(-6.5, -5)("Glitch_"+str(1)+'_'+'log10_h')
glitch_wf = models.glitch_delay(log10_h = log10_h, tau = tau, log10_f0 = log10_f0, t0 = t0, phase0 = phase0, tref=53000*86400,
                                        psr_float_idx = psr_idx, pulsars=psrs)
glitches.append(deterministic_signals.Deterministic(glitch_wf, name='Glitch'+str(1) ))

In [22]:
tm = gp_signals.TimingModel(use_svd=True)
wn = blocks.white_noise_block(vary=False, inc_ecorr=False)
#s = base_model
s = tm + wn + glitches[0]
model = []
for p in psrs:
    model.append(s(p))
with open(noise_file, 'r') as fp:
    noisedict = json.load(fp)
    pta = signal_base.PTA(model)
    pta.set_default_params(noisedict)

In [23]:
d0 = parameter.sample(pta.params)
x0 = np.array([d0[par.name] for par in pta.params])

In [24]:
pta.get_lnlikelihood(x0)

282447.35928643856

In [7]:
282055.80198464915
281999.5402583812

449134.63863715366

In [25]:
d0

{'Glitch_1_log10_f0': -8.220438412567342,
 'Glitch_1_log10_h': -5.484930699500834,
 'Glitch_1_phase0': 4.685424855153569,
 'Glitch_1_psr_idx': 2.4099188686936883,
 'Glitch_1_t0': 8.761127447495467,
 'Glitch_1_tau': 2.531727661210686}

In [26]:
import Fast_Burst_likelihood as FB

In [27]:
FB1 = FB.FastBurst(pta = pta, psrs = psrs, params = d0, Npsr = len(psrs), tref=53000*86400)
#FB2 = FB.FastBurst(pta = pta, psrs = psrs, params = d0, Npsr = len(psrs), tref=53000*86400)

Nvecs arary:  [[1.69852389e-12 1.12772641e-10 3.26328092e-12 ... 4.85655692e-12
 5.14669141e-13 1.79238956e-12], [6.95185145e-14 1.57784257e-13 2.75345324e-14 ... 1.37317696e-13
 6.53510393e-13 3.42658995e-13], [4.94236756e-10 4.03404993e-10 7.90600715e-10 ... 2.03923984e-10
 4.69194849e-10 4.02505715e-10], [1.78150121e-13 6.22555077e-13 1.41224754e-13 ... 4.44123286e-11
 2.71409648e-11 3.12885720e-11], ...]
-24156361.25289259
logdet_phi:  15289.165017480458
logdet_sigma:  4448.603620972838
dotSigmaTNr:  6171.411084360541
logdet_phi:  25604.746234093782
logdet_sigma:  8639.86438247935
dotSigmaTNr:  48950178.23814091
logdet_phi:  10683.994831492373
logdet_sigma:  2880.4602861880994
dotSigmaTNr:  1739.6367892941803
logdet_phi:  10591.89142777261
logdet_sigma:  3117.0202293660345
dotSigmaTNr:  0.5697697597075208
282055.80198464915


In [28]:
FB1.get_lnlikelihood(10**(x0[1]), x0[2], 10**(x0[0]), (3.15*10**7)*x0[5], (3.15*10**7)*x0[4], x0[3])

Amplitude:  3.273929328982241e-06
Frequency:  6.019516197895964e-09
glitch_index:  2.4099188686936883
Old Sigma:  [0. 0.]
New sigma:  [-8.82679434e-08  3.27273922e-06]
Old M and N:  [[0. 0.]
 [0. 0.]] [0. 0.]
Input time:  3194.1610485660553
Cosine:  [0. 0. 0. ... 0. 0. 0.]
Sine:  [0. 0. 0. ... 0. 0. 0.]
0.0
0.0
0.0
0.0
MM matrix: [[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
0.0
0.0
NN matrix: [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Cosine:  [0. 0. 0. ... 0. 0. 0.]
Sine:  [0. 0. 0. ... 0. 0. 0.]
0.0
0.0
0.0
0.0
MM matrix: [[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
0.0
0.0
NN matrix: [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Cosine:  [-1.34472072e-01 -1.34472072e-01 -1.34472072e-01 ... -6.27768825e-05
 -6.27768824e-05 -6.27768823e-05]
Sine:  [1.29426123e-01 1.29426123e-01 1.29426123e-01 ... 2.68815631e-06
 2.68815626e-06 2.68815623e-06]
10619574684376.854
302341398420.4929
302341398420.4925
6752715038179.544


281999.5402583812

In [12]:
482127.23089051904 #Bence version

482127.23089051904

In [13]:
448629.5379522237 #Enterprise version

448629.5379522237

In [14]:
LogL:  447940.2057596576
LogL:  447934.2907498001
LogL:  447934.2692285934
LogL:  447934.6153235128
447934.6153235128

447934.6153235128